# Mise en place des modèles de clustering

In [1]:
# Importations
import warnings
warnings.filterwarnings('ignore')
import pickle
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer

ModuleNotFoundError: No module named 'yellowbrick'

Dans cette section, nous allons réaliser un modèle de clustering qui réalisera une segmentation des clients basée sur la méthode RFM (Récence, Fréquence, Montant). Cette segmentation sépare les clients selon trois critères : la date de la dernière commande passée, le nombre de commandes passées et le montant total dépensé par le client. 

## Choix des variables et analyses multivariées

Commençons par charger notre dataframe. Les colonnes qui vont nous être utiles ici sont les colonnes total_spent, last_order et order_frequency.

In [ ]:
df = pickle.load(open("clean_data.pkl", "rb"))
cluster_df = df[["total_spent", "last_order", "orders_frequency"]]
print(cluster_df.info())

D'abord, affichons une heatmap qui représente la matrice de corrélation de la dataframe afin de voir si certaines colonnes sont corrélées :

In [ ]:
sns.heatmap(cluster_df.corr())

On constate que les corrélations entre nos différentes features sont faibles, voire nulles.

Avant de réaliser un clustering sur nos données, il est important de mieux les comprendre. Pour cela, nous allons réaliser un nuage de points en trois dimensions de notre dataframe en utilisant la technique du PCA (Principal Component Analysis). Cela nous permettra de mieux identifier les différents clusters potentiels dans nos données.

Pour bien voir l'intérêt du PCA, affichons d'abord nos données sans PCA :

In [ ]:
# Affichage du nuage de points sans PCA
fig = plt.figure()
fig.set_size_inches(27, 75)
ax = fig.add_subplot(111, projection = '3d')

x = cluster_df['total_spent']
y = cluster_df['last_order']
z = cluster_df['orders_frequency']

ax.set_xlabel("total spent")
ax.set_ylabel("last order")
ax.set_zlabel("orders frequency")

ax.scatter(x, y, z)

plt.show()

Et, maintenant, faisons la même chose en appliquant le PCA à nos données avant de les afficher :

In [ ]:
# Affichage du nuage de points avec PCA
pca = PCA(n_components=3)
standardized_cluster_df = StandardScaler().fit_transform(cluster_df)
pca_cluster = pd.DataFrame(pca.fit_transform(standardized_cluster_df))

fig = plt.figure()
fig.set_size_inches(27, 75)
ax = fig.add_subplot(111, projection = '3d')

x = pca_cluster[0]
y = pca_cluster[1]
z = pca_cluster[2]

ax.set_xlabel("total spent")
ax.set_ylabel("last order")
ax.set_zlabel("orders frequency")

ax.scatter(x, y, z)

plt.show()


Bien qu'aucun de ces deux nuages de points en 3D ne soit très lisible, on peut tout de même plus facilement distinguer des groupes de points dans le second graphique (après PCA).

Si l'on affiche maintenant la heatmap de nos données après y avoir appliqué le PCA, on constate qu'il n'y a plus de corrélation entre les différentes features. L'avantage du PCA est également d'éliminer ces colinéarités entre features, qui peuvent affecter la précision de notre modèle de clustering :

In [ ]:
sns.heatmap(pca_cluster.corr())

## Clustering

### K-means

Avant de commencer à créer nos modèles de clustering, une question se pose : combien de clusters faut-il identifier ? En effet, lorsque l'on crée un modèle, il faut lui donner le nombre de clusters à créer. Pour cela, nous pouvons utiliser la méthode elbow, qui identifie le nombre optimal de clusters. Au-delà de ce nombre, on court le risque de créer trop de clusters, qui seront trop restreints individuellement.

In [ ]:
model = KMeans()
visualizer = KElbowVisualizer(model, k=(1,20))

visualizer.fit(pca_cluster)       
visualizer.show()        

La librairie Yellowbrick, que nous avons utilisée pour générer ce graphique, nous montre le nombre de clusters optimal, 5 dans ce cas. Maintenant que nous connaissons le nombre de clusters à utiliser, nous pouvons générer notre modèle de clustering à l'aide de la méthode des k-means :

In [ ]:
clusters = KMeans(n_clusters=5).fit(pca_cluster.astype(float))


fig = plt.figure()
fig.set_size_inches(27, 75)
ax = fig.add_subplot(111, projection = '3d')

x = pca_cluster[0]
y = pca_cluster[1]
z = pca_cluster[2]

ax.set_xlabel("total spent")
ax.set_ylabel("last order")
ax.set_zlabel("orders frequency")

ax.scatter(x, y, z, c=clusters.labels_.astype(float))

plt.show()


Le nuage de points 3D obtenu n'est pas très lisible. Un bon moyen de mieux visualiser les différences entre chaque cluster obtenu est de dessiner un graphique en toile d'araignée où chaque axe correspond à une feature :

In [ ]:
k_means_cluster_df = pd.DataFrame(cluster_df)
k_means_cluster_df["cluster"] = clusters.labels_.astype(int)
# On calcule la moyenne de chaque feature à l'intérieur de chaque catégorie
categories_df = k_means_cluster_df.groupby(["cluster"]).mean()
print(categories_df)
# On normalise ensuite toutes les valeurs de la dataframe pour avoir une
# échelle cohérente dans notre graphique
categories_df = (categories_df-categories_df.mean())/categories_df.std()
categories_df.reset_index(inplace=True)
print(categories_df)
melted_df = pd.melt(categories_df, id_vars=['cluster'], var_name='feature', value_name='data',
             value_vars=['total_spent', 'last_order', 'orders_frequency'])

fig = px.line_polar(melted_df, r="data", theta="feature", color="cluster", line_close=True)
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()



Les tableaux et le graphique ci-dessus nous permettent de mettre en évidence plusieurs éléments : 
- les clients qui font partie du groupe 2 dépensent nettement plus d'argent que les autres groupes (environ 1200 dollars au total par client en moyenne). Une attention toute particulière devrait donc être prêtée aux clients assignés à ce groupe par le modèle
- Les clients du groupe 4 présentent aussi un intérêt. Ils passent commande beaucoup plus fréquemment que les autres clients et dépensent aussi beaucoup globalement (730 dollars en moyenne)
- Les clients des groupes 0, 1 et 3 ne présentent qu'un intérêt limité dans la mesure où ils dépensent relativement peu comparé aux autres.

In [ ]:
# Performances du modèle :
labels = clusters.labels_
metrics.silhouette_score(pca_cluster, labels)

In [ ]:
Classification ascendante hiérarchique (CAH)  (à venir)

In [ ]:
Dbscan (à venir)

In [ ]:
Optics (à venir)